In [2]:
import cv2
import mediapipe as mp
import time

In [36]:
cap = cv2.VideoCapture('2.mp4')  ## run our video
pTime = 0

In [37]:
mpFaceDetection = mp.solutions.face_detection
mpDraw = mp.solutions.drawing_utils
faceDetection = mpFaceDetection.FaceDetection(0.75)

In [38]:
while True:
    success, img = cap.read()
    
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  ## it must be RGB format
    results = faceDetection.process(imgRGB)
    print(results)
    
    if results.detections:
        for id, detection in enumerate(results.detections):
            # mpDraw.draw_detection(img, detection)
            # print(id, detection)
            # print(detection.score)
            # print(detection.location_data.relative_bounding_box)
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, ic = img.shape
            bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                   int(bboxC.width * iw), int(bboxC.height * ih)
            cv2.rectangle(img, bbox, (255, 0, 255), 2)
            cv2.putText(img, f'{int(detection.score[0] * 100)}%',
            (bbox[0], bbox[1] - 20), cv2.FONT_HERSHEY_PLAIN,
            2, (255, 0, 255), 2)     ## we are drawing by ourself's
    
    
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, f"FPS: {int(fps)}", (20, 70), cv2.FONT_HERSHEY_PLAIN,
                3, (0, 255, 0), 2)
    
    cv2.imshow('Image', img)
    cv2.waitKey(15)

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [63]:
## Face Detection Module



class FaceDetector():
    def __init__(self, minDetectionCon = 0.5):
        
        self.minDetectionCon = minDetectionCon
        
        self.mpFaceDetection = mp.solutions.face_detection
        self.mpDraw = mp.solutions.drawing_utils
        self.faceDetection = self.mpFaceDetection.FaceDetection(self.minDetectionCon)
        
        
    def findFaces(self, img, draw = True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  ## it must be RGB format
        self.results = self.faceDetection.process(imgRGB)
        print(self.results)
        bboxs = []
        if results.detections:
            for id, detection in enumerate(self.results.detections):
                
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, ic = img.shape
                bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                       int(bboxC.width * iw), int(bboxC.height * ih)
                bboxs.append([id, bbox, detection.score])
                if draw:
                    img = self.fancyDraw(img, bbox)
                
                cv2.putText(img, f'{int(detection.score[0] * 100)}%',
                (bbox[0], bbox[1] - 20), cv2.FONT_HERSHEY_PLAIN,
                2, (255, 0, 255), 2) 
        return img, bbox
    
    
    def fancyDraw(self, img, bbox, l = 30, t = 5, rt = 1):
        x, y, w, h = bbox
        x1, y1, = x+w, y+h ## diagnol point
        
        cv2.rectangle(img, bbox, (255, 0, 255), rt)
        
        ## top left x, y
        cv2.line(img, (x, y), (x+l,y), (255, 0, 255), t)  ## these are for the frame's corner
        cv2.line(img, (x, y), (x,y+l), (255, 0, 255), t)
        ## top right x1,y
        cv2.line(img, (x1, y), (x1-l,y), (255, 0, 255), t) ## focus the minus, it is for inside(try,check)
        cv2.line(img, (x1, y), (x1,y+l), (255, 0, 255), t)
        ## bottom left x, y1
        cv2.line(img, (x, y1), (x+l,y1), (255, 0, 255), t)  
        cv2.line(img, (x, y1), (x,y1-l), (255, 0, 255), t) ## focus the minus
        ## bottom right x1, y1
        cv2.line(img, (x1, y1), (x1-l,y1), (255, 0, 255), t)  
        cv2.line(img, (x1, y1), (x1,y1-l), (255, 0, 255), t)
        
        return img


def main():     
    cap = cv2.VideoCapture('2.mp4')
    pTime = 0
    detector = FaceDetector()
    
    while True:
        success, img = cap.read()
        img, bboxs  = detector.findFaces(img)
        print(bboxs)
        
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        cv2.putText(img, f"FPS: {int(fps)}", (20, 70), cv2.FONT_HERSHEY_PLAIN,
                    3, (0, 255, 0), 2)
        cv2.imshow('Image', img)
        cv2.waitKey(1)
    
    
if __name__ == '__main__':
    main()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
(243, 104, 171, 171)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(242, 103, 171, 171)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(241, 103, 172, 172)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(242, 104, 171, 171)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(242, 105, 171, 171)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(243, 107, 171, 171)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(244, 107, 170, 170)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(243, 107, 168, 168)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(243, 107, 168, 168)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(242, 108, 169, 169)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(241, 108, 169, 169)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(241, 109, 168, 168)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(240, 1

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
